<a href="https://colab.research.google.com/github/vamsibramhadevi/Automatic_Vehicle_NumberPlate_Detection/blob/main/NumberPlate_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/Muhammad-Zeerak-Khan/Automatic-License-Plate-Recognition-using-YOLOv8.git

Cloning into 'Automatic-License-Plate-Recognition-using-YOLOv8'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 48 (delta 10), reused 10 (delta 10), pack-reused 32
Receiving objects: 100% (48/48), 5.38 MiB | 19.91 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [27]:
!git clone https://github.com/abewley/sort.git

Cloning into 'sort'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 205 (delta 0), reused 0 (delta 0), pack-reused 202
Receiving objects: 100% (205/205), 1.20 MiB | 11.01 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [4]:
!pip install -r /content/Automatic-License-Plate-Recognition-using-YOLOv8/requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.4/595.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 23.7 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=44f49a7a796f3c19bf097a45b93e28a99fcc55729979d2d49c00c

In [ ]:
!pip install -r /content/sort/requirements.txt

In [31]:
from ultralytics import YOLO
import string
import cv2
import numpy as np
from sort.sort import *

#Load Models

In [6]:
coco_model=YOLO('yolov8n.pt')
license_plate_detector = YOLO('/content/Automatic-License-Plate-Recognition-using-YOLOv8/license_plate_detector.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 67.7MB/s]


#Load video

In [21]:
video= cv2.VideoCapture('/content/Demo.mp4')

# Read frames

In [34]:
from torch._C import S
vehicles = [2, 3, 5, 7] # pre trained class id's 2 for car and 3 for bike and 5->bus, 7-> truck vechiles
results={}
# mot_tracker = Sort()
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = video.read()
    if ret and frame_nmr<10:
        # results[frame_nmr] = {}

        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

#         # detect license plates
#         license_plates = license_plate_detector(frame)[0]
#         for license_plate in license_plates.boxes.data.tolist():
#             x1, y1, x2, y2, score, class_id = license_plate

#             # assign license plate to car
#             xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

#             if car_id != -1:

#                 # crop license plate
#                 license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

#                 # process license plate
#                 license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
#                 _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

#                 # read license plate number
#                 license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

#                 if license_plate_text is not None:
#                     results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
#                                                   'license_plate': {'bbox': [x1, y1, x2, y2],
#                                                                     'text': license_plate_text,
#                                                                     'bbox_score': score,
#                                                                     'text_score': license_plate_text_score}}

# # write results
# write_csv(results, './test.csv')

ImportError: ignored